## Web scraping

In [35]:
import requests
from bs4 import BeautifulSoup as bs
from random import randint
from time import sleep

import re

#Read the review page on British Airways from airlinequality.com
URL = 'https://www.airlinequality.com/airline-reviews/british-airways'
number_of_comments_per_pages=100
link_arguments='/?sortby=post_date%3ADesc&pagesize='
first_page_URL=URL+link_arguments+str(number_of_comments_per_pages)
req = requests.get(first_page_URL)
url_parsed = bs(req.text, 'html.parser')


#Find the number of pages
navigation = url_parsed.find("div", class_='pagination-total')
pattern = re.compile("\d+ to \d+ of (\d+) Reviews")
number_of_pages=int(int(pattern.search(navigation.text.strip()).group(1))/number_of_comments_per_pages)
print(number_of_pages)



34


In [36]:

reviews= []
ratings=[]
headers=[]
#Save first page's reviews'headers and contents and rankings
for review in url_parsed.find_all('div',attrs={'itemprop':'reviewBody'}):
    reviews.append(review.text.strip().partition('|')[2]+" ")
for header in url_parsed.find_all('h2',class_='text_header'):
    headers.append(header.text.strip()[1:-1]+" ")
#The first rating in all pages is the overall rate
rating_tags=url_parsed.find_all('span',attrs={'itemprop':'ratingValue'})
for rating in rating_tags[1:]:
    ratings.append(int(rating.text.strip())+ 0)

overall_rating = int(rating_tags[0].text.strip())
#print(reviews)
#print(headers)
print(ratings)
print(overall_rating)

[9, 9, 1, 3, 4, 5, 1, 1, 2, 8, 6, 4, 4, 3, 1, 1, 8, 3, 1, 1, 2, 2, 5, 6, 1, 8, 3, 6, 3, 4, 3, 1, 9, 2, 7, 7, 1, 4, 1, 2, 1, 5, 1, 1, 1, 6, 1, 9, 9, 3, 9, 2, 8, 1, 3, 7, 8, 9, 1, 10, 3, 6, 7, 8, 3, 9, 1, 10, 10, 4, 5, 1, 2, 2, 7, 1, 4, 1, 10, 9, 1, 1, 1, 10, 10, 1, 8, 1, 1, 2, 2, 8, 1, 1, 1, 5, 5, 7, 1, 10]
5


In [37]:
from time import sleep

for i in range(2,int(number_of_pages)):
    sleep(randint(3, 7))
    next_page=URL+'/page/'+str(i)+link_arguments+str(number_of_comments_per_pages)
    next_page_req = requests.get(next_page)
    next_page_parsed = bs(next_page_req.text, 'html.parser')
    for review in next_page_parsed.find_all('div',attrs={'itemprop':'reviewBody'}):
        reviews.append(review.text.strip().partition('|')[2]+" ")
    for header in url_parsed.find_all('h2',class_='text_header'):
        headers.append(header.text.strip()[1:-1]+" ")
    rating_tags=url_parsed.find_all('span',attrs={'itemprop':'ratingValue'})
    for rating in rating_tags[1:]:
        ratings.append(int(rating.text.strip())+0)
        
print(len(reviews))

3300


In [40]:
len(ratings)
len(headers)

3300

In [41]:
import pandas as pd
list_dict = {'Header':headers, 'Review':reviews, 'Rating':ratings} 
df = pd.DataFrame(list_dict) 
df.to_csv('British_Airways_Reviews.csv', index=False) 
